In [1]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import time
from sklearn.metrics import roc_auc_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np
import pandas as pd
import pandas.api.types
from sklearn.metrics import roc_curve, auc, roc_auc_score
import torchvision
import timm
from peft import LoraConfig, get_peft_model
from torchvision.ops import sigmoid_focal_loss
import torch.nn.functional as F

class HighTPRPartialAUCLoss(nn.Module):
    def __init__(self, tpr_threshold=0.8, lambda_rank=1.0, device='cpu'):
        super().__init__()
        self.tpr_threshold = tpr_threshold
        self.lambda_rank = lambda_rank
        self.device = device

    def forward(self, logits, targets):
        logits = logits.view(-1).to(self.device)
        targets = targets.view(-1).to(self.device)
    
        # Numerical stability
        logits = torch.clamp(logits, min=-50, max=50)
        bce_loss = F.binary_cross_entropy_with_logits(logits, targets)
    
        pos_scores = logits[targets == 1]
        neg_scores = logits[targets == 0]
    
        if len(pos_scores) == 0 or len(neg_scores) == 0:
            return bce_loss
    
        tpr_threshold_idx = min(int((1-self.tpr_threshold) * len(pos_scores)), len(pos_scores) - 1)
        pos_scores_sorted, _ = torch.sort(pos_scores)
        tpr_threshold_score = pos_scores_sorted[tpr_threshold_idx]
    
        low_tpr_pos = pos_scores[pos_scores < tpr_threshold_score]
        k = max(1, int(0.2 * len(neg_scores)))
        high_conf_neg, _ = torch.topk(neg_scores, k=k, largest=True)
    
        if len(low_tpr_pos) == 0 or len(high_conf_neg) == 0:
            return bce_loss
    
        pairwise_diff = high_conf_neg.unsqueeze(0) - low_tpr_pos.unsqueeze(1) + 1.0
        pairwise_loss = torch.clamp(pairwise_diff, min=0).mean()
    
        total_loss = bce_loss + self.lambda_rank * pairwise_loss
        return total_loss

class PartialAUCWithLogitsLoss(nn.Module):
    def __init__(self, pos_weight=1.0, lambda_conf=0.1, alpha=0.2, device='cpu'):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss(pos_weight=torch.tensor(pos_weight).to(device))
        self.lambda_conf = lambda_conf
        self.alpha = alpha
        self.device = device

    def forward(self, logits, targets):
        logits = logits.view(-1)
        targets = targets.view(-1)

        # BCE Loss (standard)
        bce_loss = self.bce(logits, targets)

        # Separate positive and negative examples
        pos_mask = targets == 1
        neg_mask = targets == 0

        pos_scores = logits[pos_mask]
        neg_scores = logits[neg_mask]

        # Rank negatives to get top α quantile (lowest FPR region)
        if len(neg_scores) == 0 or len(pos_scores) == 0:
            return bce_loss  # skip partial AUC if not enough data

        k = max(1, int(self.alpha * len(neg_scores)))
        top_neg_scores, _ = torch.topk(neg_scores, k=k, largest=False)  # lowest scores → high confidence negatives

        # Pairwise ranking loss: Hinge loss (1 + neg - pos)_+
        # All combinations between pos and top neg
        pos_scores = pos_scores.unsqueeze(1)  # shape (P, 1)
        top_neg_scores = top_neg_scores.unsqueeze(0)  # shape (1, N)
        pairwise_margin = 1.0

        pairwise_diff = top_neg_scores - pos_scores + pairwise_margin
        pairwise_loss = torch.clamp(pairwise_diff, min=0).mean()

        total_loss = bce_loss + self.lambda_conf * pairwise_loss
        return total_loss


class FocalLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        return sigmoid_focal_loss(
            inputs=inputs,                  
            targets=targets.float(),        
            alpha=self.alpha,
            gamma=self.gamma,
            reduction=self.reduction
        )

# params
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
AUG = False
LORA = True
FREEZE = True # whether or not to freeze the weights
EPOCHS = 10 # number of training epochs
DUPE = True
DUPE_FACTOR = 3
DUPE_REAL = 25
BAL = False
REG_DUPE = 100

#TRAIN_DIR = 'C:\\Users\\rngki\\Downloads\\train_images_hair_removed_dullrazor\\'
TRAIN_CSV = './og_train_dataset_with_labeled_cancer_and_skin_tone.csv' #'./new_train_dataset_with_skin_tone.csv' #
TRAIN_DIR = ['./ISIC_2024_Training_Input/ISIC_2024_Training_Input/','./ISIC_2024_Training_Input/ISIC_2024_Training_Input/']#,"C:\\Users\\rngki\\Downloads\\train_images_hair_removed_dullrazor\\"]
AUG_DIR = None #'./cleaned_styled_data/cleaned_styled_data/' #FIX
AUG_CSV = None #'./cleaned_augmented_data_with_labeled_cancer.csv' #FIX
#TRAIN_CSV = 'C:\\Users\\rngki\\Downloads\\train-metadata.csv'
#TRAIN_CSV = 'C:\\Users\\rngki\\Downloads\\cleaned_styled_data\\cleaned_styled_data\\cleaned_augmented_data.csv'
#TEST_DIR = 'C:\\Users\\rngki\\Downloads\\test_images_hair_removed_dullrazor\\'
#TEST_CSV = 'C:\\Users\\rngki\\Downloads\\test_dataset_with_skin_tone.csv'
BAL_FLAGS = [False,False]
CRIT = [HighTPRPartialAUCLoss(device=device),HighTPRPartialAUCLoss(device=device)]
#FocalLoss(alpha=0.75, gamma=2.0, reduction='mean').to(device)

MODELS = ["deit"]
#this should be ISIC_2024_Training_Input_std_LORA_FP_.75
DATA_NAMES = ['std_Fixed_pAUC','std_nolora_fixed_pAUC']# 'ISIC_2024_Training_Input_std_smart_balance', 'ISIC_2024_Training_Input_std_smart_balance', 'ISIC_2024_Training_Input_std_pAUC', 'ISIC_2024_Training_Input_std_Normal_deit']
TEST_DIR = ['./ISIC_2024_Training_Input/ISIC_2024_Training_Input/', './ISIC_2024_Training_Input/ISIC_2024_Training_Input/']
TEST_CSV = './og_test_dataset_with_labeled_cancer_and_skin_tone.csv'#'./new_test_dataset_with_skin_tone.csv' #

"""
2024 ISIC Challenge primary prize scoring metric

Given a list of binary labels, an associated list of prediction 
scores ranging from [0,1], this function produces, as a single value, 
the partial area under the receiver operating characteristic (pAUC) 
above a given true positive rate (TPR).
https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.

(c) 2024 Nicholas R Kurtansky, MSKCC
"""



class ParticipantVisibleError(Exception):
    pass

def initialize_model(model_name, num_classes=1):
    model = None
    
    if model_name == "efficientnet":
        target_modules=[
            # final linear
            "classifier.1",
            # a couple of the deepest MBConv depthwise convs:
            "features.6.block.2.conv",  
            "features.7.block.2.conv",  
        ]
        model = torchvision.models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        num_ftrs = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_ftrs, num_classes)
        final_layer = model.classifier[1]
    
    elif model_name == "resnet34":
        target_modules=[
            # final classification
            "fc",
            # pick one conv in each of the last 3 residual stages:
            "layer2.0.conv1",
            "layer3.1.conv2",
            "layer4.2.conv2",
        ]
        model = torchvision.models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
        final_layer = model.fc
    
    elif model_name == "resnet50":
        target_modules=[
            # final classification
            "fc",
            # pick one conv in each of the last 3 residual stages:
            "layer2.0.conv1",
            "layer3.1.conv2",
            "layer4.2.conv2",
        ]
        model = torchvision.models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
        final_layer = model.fc
    
    elif model_name == "coatnet":
        target_modules=[
            # final linear
            "head.fc",
            # point‑wise & depth‑wise conv in the last stage:
            "stages.3.blocks.2.pwconv",
            "stages.3.blocks.2.conv",  
        ]
        model = timm.create_model('coatnet_0_rw_224', pretrained=True)
        num_ftrs = model.head.fc.in_features
        model.head.fc = nn.Linear(num_ftrs, num_classes)
        final_layer = model.head.fc
    
    elif model_name == "deit":
        target_modules=[
            # QKV + output projection in each of the first few heads:
            "blocks.0.attn.qkv",   "blocks.0.attn.proj",
            "blocks.1.attn.qkv",   "blocks.1.attn.proj",
            # …repeat for as many blocks as you like…
            # final classifier(s):
            "head", 
            "head_dist",         # only for the distilled variant
        ]
        model = timm.create_model('deit_base_patch16_224', pretrained=True)
        num_ftrs = model.head.in_features
        model.head = nn.Linear(num_ftrs, num_classes)
        final_layer = model.head
    
    else:
        raise ValueError("Invalid model name")
    
    # Freeze all layers
    if FREEZE:
        for param in model.parameters():
            param.requires_grad = False
        # Unfreeze only the final classification layer
        for param in final_layer.parameters():
            param.requires_grad = True
    if LORA:
        lora_config = LoraConfig(
            r=1,
            lora_alpha=2,
            lora_dropout=0.1,
            target_modules=target_modules,
        )
    
        model = get_peft_model(model, lora_config)
    
    return model.to(device)

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80) -> float:
    '''
    2024 ISIC Challenge metric: pAUC
    
    Given a solution file and submission file, this function returns the
    the partial area under the receiver operating characteristic (pAUC) 
    above a given true positive rate (TPR) = 0.80.
    https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.
    
    (c) 2024 Nicholas R Kurtansky, MSKCC

    Args:
        solution: ground truth pd.DataFrame of 1s and 0s
        submission: solution dataframe of predictions of scores ranging [0, 1]

    Returns:
        Float value range [0, max_fpr]
    '''
    for col in solution.columns:
        if col != 'is_malignant':
            del solution[col]
    
    for col in submission.columns:
        if col != 'prediction':
            del submission[col]

    # check submission is numeric
    if not pandas.api.types.is_numeric_dtype(submission.values):
        raise ParticipantVisibleError('Submission target column must be numeric')

    # rescale the target. set 0s to 1s and 1s to 0s (since sklearn only has max_fpr)
    v_gt = abs(solution.values.ravel()-1)
    
    # flip the submissions to their compliments
    v_pred = -1.0*submission.values.ravel()

    max_fpr = abs(1-min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)
        
    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)

#     # Equivalent code that uses sklearn's roc_auc_score
#     v_gt = abs(np.asarray(solution.values)-1)
#     v_pred = np.array([1.0 - x for x in submission.values])
#     max_fpr = abs(1-min_tpr)
#     partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
#     # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
#     # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
#     partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return(partial_auc)

for dir in range(len(TRAIN_DIR)):
    BAL = BAL_FLAGS[dir]
    USE_SPLIT = False

    MALIGNANT = 'target'
    # MALIGNANT = 'target'
    MALIG_IDX = 1
    # MALIG_IDX = 3

    if not TEST_DIR:
        TEST_DIR = TRAIN_DIR[dir]
        USE_SPLIT = True
    if not TEST_CSV:
        TEST_CSV = TRAIN_CSV
        USE_SPLIT = True

    # def balance_classes(df, label_col=MALIGNANT):
    #     # Separate malignant and non-malignant samples
    #     malig_df = df[df[label_col] == 1]
    #     non_malig_df = df[df[label_col] == 0]

    #     # Get the minority count
    #     min_count = min(len(malig_df), len(non_malig_df))

    #     # Downsample both classes to min_count (or oversample malignant if needed)
    #     non_malig_df_balanced = non_malig_df.sample(n=min_count, random_state=42)
    #     malig_df_balanced = malig_df.sample(n=min_count, replace=True, random_state=42)  # oversample if needed

    #     # Concatenate back
    #     balanced_df = pd.concat([malig_df_balanced, non_malig_df_balanced]).sample(frac=1, random_state=42).reset_index(drop=True)
    #     return balanced_df

    if USE_SPLIT:
        df = pd.read_csv(TRAIN_CSV)
        #df = pd.read_csv('C:\\Users\\rngki\\Downloads\\cleaned_styled_data\\cleaned_styled_data\\cleaned_augmented_data.csv')
        print(f"num malignant: {sum(df[MALIGNANT])}")

        # downsample for time
        df = df.sample(frac=.1, random_state=42)

        train_df, val_df = train_test_split(df, test_size=0.2, stratify=df[MALIGNANT], random_state=42)
    elif AUG:
        OG_train_df = pd.read_csv(TRAIN_CSV)
        AUG_train = pd.read_csv(AUG_CSV)
        # Add a source column to each
        OG_train_df["source"] = 0
        AUG_train["source"] = 1
        
        #SOURCE_IDX = AUG_train.columns.get_loc("source")

# Concatenate the two DataFrames
        train_df = pd.concat([OG_train_df, AUG_train], ignore_index=True)
        val_df = pd.read_csv(TEST_CSV)
    else:
        train_df = pd.read_csv(TRAIN_CSV)
        val_df = pd.read_csv(TEST_CSV)

    if DUPE:

        if AUG:
            real_malig_df = train_df[(train_df[MALIGNANT] == 1) & (train_df['source'] == 0)]
            fake_malig_df = train_df[(train_df[MALIGNANT] == 1) & (train_df['source'] == 1)]
            
            real_malig_df = pd.concat([real_malig_df]*DUPE_REAL*DUPE_FACTOR, ignore_index=True)
            fake_malig_df = pd.concat([fake_malig_df]*DUPE_FACTOR, ignore_index=True)
    
            train_df = pd.concat([train_df, real_malig_df,fake_malig_df], ignore_index=True)
    
            if BAL:
                # Step 1: Separate malignant and non-malignant
                malig_df = train_df[train_df[MALIGNANT] == 1]
                
                non_malig_df = train_df[train_df[MALIGNANT] == 0]
                
                # Step 2: Balance malignant skin tones within each source
                balanced_by_source = []
                for src in [0, 1]:
                    sub_df = malig_df[malig_df["source"] == src]
                
                    if sub_df.empty:
    
                        print("problem")
                        continue  # Skip if no data for this source
                
                    # Get max count for balancing
                    skin_counts = sub_df["skin_tone"].value_counts()
                    max_count = skin_counts.max()
                
                    # Duplicate each skin tone group to match max_count
                    balanced_sub = (
                        sub_df.groupby("skin_tone", group_keys=False)
                        .apply(lambda x: x.sample(max_count, replace=True))
                    )
                    balanced_by_source.append(balanced_sub)
                
                # Step 3: Combine balanced malignant data
                malig_df_balanced = pd.concat(balanced_by_source).reset_index(drop=True)
                
                # Step 4: Combine with non-malignant data and shuffle
                train_df= pd.concat([non_malig_df, malig_df_balanced], ignore_index=True)
                train_df = train_df.sample(frac=1).reset_index(drop=True)
        else:
            malig_df = train_df[train_df[MALIGNANT] == 1]
            malig_df = pd.concat([malig_df]*REG_DUPE, ignore_index=True)
            
            train_df = pd.concat([train_df, malig_df], ignore_index=True)
            train_df["source"] = 0
            if BAL:
                # Step 1: Separate malignant and non-malignant
                malig_df = train_df[train_df[MALIGNANT] == 1]
                
                non_malig_df = train_df[train_df[MALIGNANT] == 0]
                
                # Step 2: Balance malignant skin tones within each source
                balanced_by_source = []
                for src in [0, 1]:
                    sub_df = malig_df[malig_df["source"] == src]
                
                    if sub_df.empty:
    
                        print("problem")
                        continue  # Skip if no data for this source
                
                    # Get max count for balancing
                    skin_counts = sub_df["skin_tone"].value_counts()
                    max_count = skin_counts.max()
                
                    # Duplicate each skin tone group to match max_count
                    balanced_sub = (
                        sub_df.groupby("skin_tone", group_keys=False)
                        .apply(lambda x: x.sample(max_count, replace=True))
                    )
                    balanced_by_source.append(balanced_sub)
                
                # Step 3: Combine balanced malignant data
                malig_df_balanced = pd.concat(balanced_by_source).reset_index(drop=True)
                
                # Step 4: Combine with non-malignant data and shuffle
                train_df= pd.concat([non_malig_df, malig_df_balanced], ignore_index=True)
                train_df = train_df.sample(frac=1).reset_index(drop=True)


    train_df.to_csv('train_labels.csv', index=False)
    val_df.to_csv('val_labels.csv', index=False)

    print(f"Validation - Num malignant: {val_df[MALIGNANT].sum()}")
    print(f"Validation - Num benign: {len(val_df) - val_df[MALIGNANT].sum()}")
    print(f"Training - Num malignant: {train_df[MALIGNANT].sum()}")
    print(f"Training - Num benign: {len(train_df) - train_df[MALIGNANT].sum()}")

    class ISICDataset(Dataset):
        def __init__(self, csv_file, img_dir, non_malignant_transform=None, malignant_transform=None,label_col=MALIGNANT,aug_dir = None, train_bool = False,Balance= False):
            self.fulldata = pd.read_csv(csv_file)
            self.train = train_bool
            self.balanced = Balance
            if self.train:
                self.malig_df = self.fulldata[self.fulldata[label_col] ==1]
                self.non_malig_df = self.fulldata[self.fulldata[label_col] ==0]
                self.min_count = min(len(self.malig_df), len(self.non_malig_df))
                non_malig_df_balanced = self.non_malig_df.sample(n=self.min_count)

                # Concatenate back
                self.df = pd.concat([self.malig_df, non_malig_df_balanced]).sample(frac=1).reset_index(drop=True)
                if aug_dir != None:
                    self.SOURCE_IDX = self.df.columns.get_loc("source")
            else:
                self.df = self.fulldata

            if self.balanced:
                self.skin_idx = self.df.columns.get_loc("skin_tone")
            
            self.img_dir = img_dir
            self.aug_dir = aug_dir
            self.non_malignant_transform = non_malignant_transform
            self.malignant_transform = malignant_transform

        def rebalance(self):
            if not self.train:
                return
            
            if (self.balanced):
                malig_skin_tones = set(self.malig_df["skin_tone"].unique())
                nonmalig_skin_tones = set(self.non_malig_df["skin_tone"].unique())
                
                exclusive_nonmalig_skin_tones = nonmalig_skin_tones - malig_skin_tones
                extra_sample_frac = 0.05 # 5% of total non_malig_df

                # Filter only the exclusive skin tones
                nonmalig_extras = self.non_malig_df[self.non_malig_df["skin_tone"].isin(exclusive_nonmalig_skin_tones)]
                
                # Sample a small subset
                nonmalig_extras_sampled = nonmalig_extras.sample(int(extra_sample_frac*self.min_count), replace=True)
                
                # Get skin_tone distribution in malignant data
                malig_skin_dist = self.malig_df["skin_tone"].value_counts(normalize=True)

                # Sample from non-malignant data using the same proportions
                non_malig_df_balanced = (
                    self.non_malig_df.groupby("skin_tone", group_keys=False)
                    .apply(lambda x: x.sample(int((1-extra_sample_frac)*self.min_count * malig_skin_dist.get(x.name, 0)), replace=True))
                )

                # Combine and shuffle
                self.df = pd.concat([self.malig_df, non_malig_df_balanced]).sample(frac=1).reset_index(drop=True)
            else:
                non_malig_df_balanced = self.non_malig_df.sample(n=self.min_count)
                # Concatenate back
                self.df = pd.concat([self.malig_df, non_malig_df_balanced]).sample(frac=1).reset_index(drop=True)



        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):
            if(self.aug_dir == None):
                img_path = os.path.join(self.img_dir, self.df.iloc[idx, 0] + '.jpg')

                image = Image.open(img_path).convert('RGB')

                if self.df.iloc[idx, MALIG_IDX] == 0 and self.non_malignant_transform:
                    image = self.non_malignant_transform(image)
                elif self.df.iloc[idx, MALIG_IDX] == 1 and self.malignant_transform:
                    image = self.malignant_transform(image)

                return image, int(self.df.iloc[idx, MALIG_IDX]), self.df.iloc[idx, 0]
            else:
                if(self.df.iloc[idx, self.SOURCE_IDX] == 0):

                    img_path = os.path.join(self.img_dir, self.df.iloc[idx, 0] + '.jpg')

                    image = Image.open(img_path).convert('RGB')

                    if self.df.iloc[idx, MALIG_IDX] == 0 and self.non_malignant_transform:
                        image = self.non_malignant_transform(image)
                    elif self.df.iloc[idx, MALIG_IDX] == 1 and self.malignant_transform:
                        image = self.malignant_transform(image)

                    return image, int(self.df.iloc[idx, MALIG_IDX]), self.df.iloc[idx, 0] 
                else:
                    img_path = os.path.join(self.aug_dir, self.df.iloc[idx, 0] + '.jpg')

                    image = Image.open(img_path).convert('RGB')

                    if self.df.iloc[idx, MALIG_IDX] == 0 and self.non_malignant_transform:
                        image = self.non_malignant_transform(image)
                    elif self.df.iloc[idx, MALIG_IDX] == 1 and self.malignant_transform:
                        image = self.malignant_transform(image)

                    return image, int(self.df.iloc[idx, MALIG_IDX]), self.df.iloc[idx, 0] 



    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet means
                            std=[0.229, 0.224, 0.225])    # ImageNet stds
    ])

    # malignant_transform = transforms.Compose([
    #     transforms.Resize((224, 224)),
    #     transforms.RandomHorizontalFlip(p=0.5),
    #     transforms.RandomVerticalFlip(p=0.5),
    #     transforms.ToTensor(),
    #     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    # ])
    malignant_transform = transforms.Compose([
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.RandomRotation(degrees=360),  # Rotate by a random degree
        transforms.RandomResizedCrop(size=224, scale=(0.8, 1.0)),  # Random crop and resize to 224x224
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])



    val_dataset = ISICDataset(
        csv_file='val_labels.csv',
        img_dir=TEST_DIR[dir],
        non_malignant_transform=transform,
        malignant_transform=transform
    )

    train_dataset = ISICDataset(
        csv_file='train_labels.csv',
        img_dir=TRAIN_DIR[dir],
        non_malignant_transform=malignant_transform,
        malignant_transform=malignant_transform,
        aug_dir = AUG_DIR,
        train_bool = True,
        Balance= BAL
    )


    
    print(f"Training on {'cuda' if torch.cuda.is_available() else 'cpu'}")

    for model_name in MODELS:
        resnet = initialize_model(model_name=model_name, num_classes=1)

        optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=1e-4)
        criterion = CRIT[dir]
        #pos_weight = torch.tensor([1.0]).to(device)
        #criterion = ConfidentBCEWithLogitsLoss(pos_weight=pos_weight, lambda_conf=0.1, device=device)
        #criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.0]).to(device))
        #criterion = PartialAUCWithLogitsLoss(pos_weight=5, lambda_conf=1, device='cuda')

        # switch to this if we're doing more than malignant/not-malignant
        #criterion = nn.CrossEntropyLoss()

        resnet.to(device)

        with open(f"training_log_{model_name}_{DATA_NAMES[dir]}_{dir}.csv", "w") as f:
            f.write("epoch,train_loss,val_loss,val_tn,val_fn,val_tp,val_fp,val_accuracy,val_auc,pauc,f1score\n")
            start_time = time.time()
            for epoch in range(EPOCHS):
                train_dataset.rebalance()
                dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
                valloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

                print(f"Training set size: {len(dataloader.dataset)}")
                print(f"Validation set size: {len(valloader.dataset)}")

                resnet.train()
                avgloss = 0.0
                print(f"EPOCH: {epoch + 1}")
                for images, labels, _ in dataloader:
                    images, labels = images.to(device), labels.float().unsqueeze(1).to(device)

                    optimizer.zero_grad()
                    outputs = resnet(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    avgloss += loss.item() * images.size(0)
                avgloss /= len(dataloader.dataset)

                resnet.eval()
                val_loss = 0.0
                total = 0
                false_negative = 0
                false_positive = 0
                true_negative = 0
                true_positive = 0
                all_labels = []
                all_probs = []  # Store probabilities instead of binary predictions
                image_ids_list = []  # Collect image IDs for submission

                with torch.no_grad():
                    for images, labels, image_ids in valloader:  # Ensure dataset returns image IDs
                        images = images.to(device)
                        labels = labels.to(device).float().unsqueeze(1)
                        
                        # Forward pass
                        outputs = resnet(images)
                        loss = criterion(outputs, labels)
                        val_loss += loss.item() * images.size(0)

                        # Get probabilities (before thresholding)
                        probabilities = torch.sigmoid(outputs).cpu().numpy().flatten()
                        all_probs.extend(probabilities)
                        
                        # Get binary predictions for confusion matrix
                        preds = (torch.sigmoid(outputs) > 0.5).float()
                        preds = preds.cpu().numpy().flatten()
                        
                        # Track image IDs for submission
                        image_ids_list.extend(image_ids)  # Assuming image_ids are strings
                        
                        # Update confusion matrix
                        for p, l in zip(preds, labels.cpu().numpy()):
                            if p == 0 and l == 0:
                                true_negative += 1
                            elif p == 0 and l == 1:
                                false_negative += 1
                            elif p == 1 and l == 0:
                                false_positive += 1
                            elif p == 1 and l == 1:
                                true_positive += 1
                        total += labels.size(0)
                        all_labels.extend(labels.cpu().numpy().flatten())

                # Calculate metrics
                avg_val_loss = val_loss / len(valloader.dataset)
                accuracy = (true_positive + true_negative) / total
                p_auc = roc_auc_score(all_labels, all_probs)  # Use probabilities for AUC
                f1score = f1_score(all_labels, [1 if x > 0.5 else 0 for x in all_probs])

                # Create submission DataFrame
                submission_df = pd.DataFrame({
                    'isic_id': image_ids_list,
                    'prediction': all_probs
                })

                solution_df = pd.DataFrame({
                    'isic_id': image_ids_list,
                    'is_malignant': all_labels
                })

                # Calculate pAUC using the competition metric
                try:
                    pAUC = score(
                        solution=solution_df,
                        submission=submission_df,
                        row_id_column_name='isic_id'  # Must match your column name
                    )
                except ParticipantVisibleError as e:
                    print(f"Scoring Error: {e}")
                    pAUC = -1  # Handle invalid submissions

                if (epoch+1) % 5 == 0:
                    torch.save(resnet.state_dict(), f"resnet50_{epoch}_{DUPE}_{FREEZE}_pos_weight(2).pth")
                
                print(f"Epoch [{epoch+1}/{EPOCHS}] "
                    f"Train Loss: {avgloss:.4f} "
                    f"Val Loss: {avg_val_loss:.4f}\n"
                    f"Val TN: {true_negative} FN: {false_negative} \n"
                    f"TP: {true_positive} FP: {false_positive}\n"
                    f"Val Accuracy: {accuracy:.4f}\n"
                    f"AUC: {p_auc:.4f}\n"
                    f"pAUC: {pAUC:.4f}\n"
                    f"f1 score: {f1score:.4f}\n")  # Add pAUC to output'
                f.write(f"{epoch+1},{avgloss},{avg_val_loss},{true_negative},{false_negative},{true_positive},{false_positive},{accuracy},{p_auc},{pAUC},{f1score}\n")
                with open(f"training_log_{model_name}_{DATA_NAMES[dir]}_{dir}live.csv", "w") as f1:
                    f1.write(f"{epoch+1},{avgloss},{avg_val_loss},{true_negative},{false_negative},{true_positive},{false_positive},{accuracy},{p_auc},{pAUC},{f1score}\n")
                with open(f"scores_{model_name}_{DATA_NAMES[dir]}_{dir}.csv", "w") as f1:
                    f1.write("isic_id,prediction,target\n")
                    for id, fscore, actual in zip(image_ids_list, all_probs, all_labels):
                        f1.write(f"{id},{fscore},{actual}\n")
            
        print(f"Training time: {time.time() - start_time:.2f}s")

        with open(f"scores_{model_name}_{DATA_NAMES[dir]}_{dir}.csv", "w") as f:
            f.write("isic_id,prediction,target\n")
            for id, fscore, actual in zip(image_ids_list, all_probs, all_labels):
                f.write(f"{id},{fscore},{actual}\n")

        

        def plot_roc_and_pauc(y_true, y_scores, min_tpr=0.80):
            # Flip labels and predictions (based on your scoring function)
            v_gt = abs(np.asarray(y_true) - 1)
            v_pred = -1.0 * np.asarray(y_scores)

            # Calculate ROC curve
            fpr, tpr, thresholds = roc_curve(v_gt, v_pred)

            # Compute full AUC and partial AUC
            full_auc = auc(fpr, tpr)
            max_fpr = abs(1 - min_tpr)

            # Partial AUC calculation (manual interpolation for the cutoff point)
            stop = np.searchsorted(fpr, max_fpr, "right")
            x_interp = [fpr[stop - 1], fpr[stop]]
            y_interp = [tpr[stop - 1], tpr[stop]]
            
            # Interpolated TPR at max_fpr
            interp_tpr = np.interp(max_fpr, x_interp, y_interp)
            
            # Create partial ROC arrays up to max_fpr
            pauc_fpr = np.append(fpr[:stop], max_fpr)
            pauc_tpr = np.append(tpr[:stop], interp_tpr)
            
            partial_auc = auc(pauc_fpr, pauc_tpr)

            # Plotting
            plt.figure(figsize=(8, 6))
            plt.plot(fpr, tpr, label=f"ROC curve (AUC = {full_auc:.4f})")
            
            # Shade pAUC region
            plt.fill_between(pauc_fpr, pauc_tpr, step="post", alpha=0.3, color="orange", label=f"pAUC = {partial_auc:.4f}")
            
            # Draw horizontal line at TPR = min_tpr
            plt.axhline(min_tpr, color='red', linestyle='--', label=f"Min TPR = {min_tpr}")

            # Draw vertical line at FPR = max_fpr
            plt.axvline(max_fpr, color='green', linestyle='--', label=f"Max FPR = {max_fpr:.2f}")

            plt.xlabel("False Positive Rate (FPR)")
            plt.ylabel("True Positive Rate (TPR)")
            plt.title("ROC Curve with pAUC Region Highlighted")
            plt.legend(loc="lower right")
            plt.grid(True)
            plt.savefig(f"roc_curve_{model_name}_{DATA_NAMES[dir]}_{dir}.png")

        # Example usage
        plot_roc_and_pauc(all_labels, all_probs)

/tmp/ipykernel_963737/340142890.py:384: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(TRAIN_CSV)


Validation - Num malignant: 81
Validation - Num benign: 80133
Training - Num malignant: 31512
Training - Num benign: 320533


/tmp/ipykernel_963737/340142890.py:482: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  self.fulldata = pd.read_csv(csv_file)


Training on cuda
Training set size: 63024
Validation set size: 80214
EPOCH: 1
Epoch [1/10] Train Loss: 1.9855 Val Loss: 0.6968
Val TN: 38152 FN: 19 
TP: 62 FP: 41981
Val Accuracy: 0.4764
AUC: 0.7231
pAUC: 0.0544
f1 score: 0.0029

Training set size: 63024
Validation set size: 80214
EPOCH: 2


KeyboardInterrupt: 

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import time
from sklearn.metrics import roc_auc_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np
import pandas as pd
import pandas.api.types
from sklearn.metrics import roc_curve, auc, roc_auc_score
import torchvision
import timm
from peft import LoraConfig, get_peft_model
from torchvision.ops import sigmoid_focal_loss

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.75, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        return sigmoid_focal_loss(
            inputs=inputs,                  
            targets=targets.float(),        
            alpha=self.alpha,
            gamma=self.gamma,
            reduction=self.reduction
        )

# params
AUG = True
LORA = True
FREEZE = True # whether or not to freeze the weights
EPOCHS = 15 # number of training epochs
DUPE = True
DUPE_FACTOR = 4
DUPE_REAL = 12
BAL = False
#TRAIN_DIR = 'C:\\Users\\rngki\\Downloads\\train_images_hair_removed_dullrazor\\'
TRAIN_CSV = './og_train_dataset_with_labeled_cancer_and_skin_tone.csv'
TRAIN_DIR = ['./ISIC_2024_Training_Input/ISIC_2024_Training_Input/'] #,"C:\\Users\\rngki\\Downloads\\train_images_hair_removed_dullrazor\\"]
AUG_DIR = './cleaned_styled_data/cleaned_styled_data/' #FIX
AUG_CSV = './cleaned_augmented_data_with_labeled_cancer.csv' #FIX
#TRAIN_CSV = 'C:\\Users\\rngki\\Downloads\\train-metadata.csv'
#TRAIN_CSV = 'C:\\Users\\rngki\\Downloads\\cleaned_styled_data\\cleaned_styled_data\\cleaned_augmented_data.csv'
#TEST_DIR = 'C:\\Users\\rngki\\Downloads\\test_images_hair_removed_dullrazor\\'
#TEST_CSV = 'C:\\Users\\rngki\\Downloads\\test_dataset_with_skin_tone.csv'

MODELS = ["deit"]

DATA_NAMES = ['ISIC_2024_Training_Input_ST_LORA_DEIT_FOCAL9']
TEST_DIR = ['./ISIC_2024_Training_Input/ISIC_2024_Training_Input/']
TEST_CSV = './og_test_dataset_with_labeled_cancer_and_skin_tone.csv'

"""
2024 ISIC Challenge primary prize scoring metric

Given a list of binary labels, an associated list of prediction 
scores ranging from [0,1], this function produces, as a single value, 
the partial area under the receiver operating characteristic (pAUC) 
above a given true positive rate (TPR).
https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.

(c) 2024 Nicholas R Kurtansky, MSKCC
"""



class ParticipantVisibleError(Exception):
    pass

def initialize_model(model_name, num_classes=1):
    model = None
    
    if model_name == "efficientnet":
        target_modules=[
            # final linear
            "classifier.1",
            # a couple of the deepest MBConv depthwise convs:
            "features.6.block.2.conv",  
            "features.7.block.2.conv",  
        ]
        model = torchvision.models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        num_ftrs = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_ftrs, num_classes)
        final_layer = model.classifier[1]
    
    elif model_name == "resnet34":
        target_modules=[
            # final classification
            "fc",
            # pick one conv in each of the last 3 residual stages:
            "layer2.0.conv1",
            "layer3.1.conv2",
            "layer4.2.conv2",
        ]
        model = torchvision.models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
        final_layer = model.fc
    
    elif model_name == "resnet50":
        target_modules=[
            # final classification
            "fc",
            # pick one conv in each of the last 3 residual stages:
            "layer2.0.conv1",
            "layer3.1.conv2",
            "layer4.2.conv2",
        ]
        model = torchvision.models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
        final_layer = model.fc
    
    elif model_name == "coatnet":
        target_modules=[
            # final linear
            "head.fc",
            # point‑wise & depth‑wise conv in the last stage:
            "stages.3.blocks.2.pwconv",
            "stages.3.blocks.2.conv",  
        ]
        model = timm.create_model('coatnet_0_rw_224', pretrained=True)
        num_ftrs = model.head.fc.in_features
        model.head.fc = nn.Linear(num_ftrs, num_classes)
        final_layer = model.head.fc
    
    elif model_name == "deit":
        target_modules=[
            # QKV + output projection in each of the first few heads:
            "blocks.0.attn.qkv",   "blocks.0.attn.proj",
            "blocks.1.attn.qkv",   "blocks.1.attn.proj",
            # …repeat for as many blocks as you like…
            # final classifier(s):
            "head", 
            "head_dist",         # only for the distilled variant
        ]
        model = timm.create_model('deit_base_patch16_224', pretrained=True)
        num_ftrs = model.head.in_features
        model.head = nn.Linear(num_ftrs, num_classes)
        final_layer = model.head
    
    else:
        raise ValueError("Invalid model name")
    
    # Freeze all layers
    if FREEZE:
        for param in model.parameters():
            param.requires_grad = False
        # Unfreeze only the final classification layer
        for param in final_layer.parameters():
            param.requires_grad = True
    if LORA:
        lora_config = LoraConfig(
            r=1,
            lora_alpha=2,
            lora_dropout=0.1,
            target_modules=target_modules,
        )
    
        model = get_peft_model(model, lora_config)
    
    return model.to(device)

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80) -> float:
    '''
    2024 ISIC Challenge metric: pAUC
    
    Given a solution file and submission file, this function returns the
    the partial area under the receiver operating characteristic (pAUC) 
    above a given true positive rate (TPR) = 0.80.
    https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.
    
    (c) 2024 Nicholas R Kurtansky, MSKCC

    Args:
        solution: ground truth pd.DataFrame of 1s and 0s
        submission: solution dataframe of predictions of scores ranging [0, 1]

    Returns:
        Float value range [0, max_fpr]
    '''
    for col in solution.columns:
        if col != 'is_malignant':
            del solution[col]
    
    for col in submission.columns:
        if col != 'prediction':
            del submission[col]

    # check submission is numeric
    if not pandas.api.types.is_numeric_dtype(submission.values):
        raise ParticipantVisibleError('Submission target column must be numeric')

    # rescale the target. set 0s to 1s and 1s to 0s (since sklearn only has max_fpr)
    v_gt = abs(solution.values.ravel()-1)
    
    # flip the submissions to their compliments
    v_pred = -1.0*submission.values.ravel()

    max_fpr = abs(1-min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)
        
    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)

#     # Equivalent code that uses sklearn's roc_auc_score
#     v_gt = abs(np.asarray(solution.values)-1)
#     v_pred = np.array([1.0 - x for x in submission.values])
#     max_fpr = abs(1-min_tpr)
#     partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
#     # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
#     # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
#     partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return(partial_auc)

for dir in range(len(TRAIN_DIR)):
    USE_SPLIT = False

    MALIGNANT = 'target'
    # MALIGNANT = 'target'
    MALIG_IDX = 1
    # MALIG_IDX = 3

    if not TEST_DIR:
        TEST_DIR = TRAIN_DIR[dir]
        USE_SPLIT = True
    if not TEST_CSV:
        TEST_CSV = TRAIN_CSV
        USE_SPLIT = True

    # def balance_classes(df, label_col=MALIGNANT):
    #     # Separate malignant and non-malignant samples
    #     malig_df = df[df[label_col] == 1]
    #     non_malig_df = df[df[label_col] == 0]

    #     # Get the minority count
    #     min_count = min(len(malig_df), len(non_malig_df))

    #     # Downsample both classes to min_count (or oversample malignant if needed)
    #     non_malig_df_balanced = non_malig_df.sample(n=min_count, random_state=42)
    #     malig_df_balanced = malig_df.sample(n=min_count, replace=True, random_state=42)  # oversample if needed

    #     # Concatenate back
    #     balanced_df = pd.concat([malig_df_balanced, non_malig_df_balanced]).sample(frac=1, random_state=42).reset_index(drop=True)
    #     return balanced_df

    if USE_SPLIT:
        df = pd.read_csv(TRAIN_CSV)
        #df = pd.read_csv('C:\\Users\\rngki\\Downloads\\cleaned_styled_data\\cleaned_styled_data\\cleaned_augmented_data.csv')
        print(f"num malignant: {sum(df[MALIGNANT])}")

        # downsample for time
        df = df.sample(frac=.1, random_state=42)

        train_df, val_df = train_test_split(df, test_size=0.2, stratify=df[MALIGNANT], random_state=42)
    elif AUG:
        OG_train_df = pd.read_csv(TRAIN_CSV)
        AUG_train = pd.read_csv(AUG_CSV)
        # Add a source column to each
        OG_train_df["source"] = 0
        AUG_train["source"] = 1
        
        #SOURCE_IDX = AUG_train.columns.get_loc("source")

# Concatenate the two DataFrames
        train_df = pd.concat([OG_train_df, AUG_train], ignore_index=True)
        val_df = pd.read_csv(TEST_CSV)
    else:
        train_df = pd.read_csv(TRAIN_CSV)
        val_df = pd.read_csv(TEST_CSV)

    if DUPE:

        if AUG:
            real_malig_df = train_df[(train_df[MALIGNANT] == 1) & (train_df['source'] == 0)]
            fake_malig_df = train_df[(train_df[MALIGNANT] == 1) & (train_df['source'] == 1)]
            
            real_malig_df = pd.concat([real_malig_df]*DUPE_REAL*DUPE_FACTOR, ignore_index=True)
            fake_malig_df = pd.concat([fake_malig_df]*DUPE_FACTOR, ignore_index=True)
    
            train_df = pd.concat([train_df, real_malig_df,fake_malig_df], ignore_index=True)
    
            if BAL:
                # Step 1: Separate malignant and non-malignant
                malig_df = train_df[train_df[MALIGNANT] == 1]
                
                non_malig_df = train_df[train_df[MALIGNANT] == 0]
                
                # Step 2: Balance malignant skin tones within each source
                balanced_by_source = []
                for src in [0, 1]:
                    sub_df = malig_df[malig_df["source"] == src]
                
                    if sub_df.empty:
    
                        print("problem")
                        continue  # Skip if no data for this source
                
                    # Get max count for balancing
                    skin_counts = sub_df["skin_tone"].value_counts()
                    max_count = skin_counts.max()
                
                    # Duplicate each skin tone group to match max_count
                    balanced_sub = (
                        sub_df.groupby("skin_tone", group_keys=False)
                        .apply(lambda x: x.sample(max_count, replace=True))
                    )
                    balanced_by_source.append(balanced_sub)
                
                # Step 3: Combine balanced malignant data
                malig_df_balanced = pd.concat(balanced_by_source).reset_index(drop=True)
                
                # Step 4: Combine with non-malignant data and shuffle
                train_df= pd.concat([non_malig_df, malig_df_balanced], ignore_index=True)
                train_df = train_df.sample(frac=1).reset_index(drop=True)
        else:
            malig_df = train_df[train_df[MALIGNANT] == 1]
            malig_df = pd.concat([malig_df]*100, ignore_index=True)
            
            train_df = pd.concat([train_df, malig_df], ignore_index=True)
            train_df["source"] = 0


    train_df.to_csv('train_labels.csv', index=False)
    val_df.to_csv('val_labels.csv', index=False)

    print(f"Validation - Num malignant: {val_df[MALIGNANT].sum()}")
    print(f"Validation - Num benign: {len(val_df) - val_df[MALIGNANT].sum()}")
    print(f"Training - Num malignant: {train_df[MALIGNANT].sum()}")
    print(f"Training - Num benign: {len(train_df) - train_df[MALIGNANT].sum()}")

    class ISICDataset(Dataset):
        def __init__(self, csv_file, img_dir, non_malignant_transform=None, malignant_transform=None,label_col=MALIGNANT,aug_dir = None, train_bool = False,Balance= False):
            self.fulldata = pd.read_csv(csv_file)
            self.train = train_bool
            self.balanced = Balance
            if self.train:
                self.malig_df = self.fulldata[self.fulldata[label_col] ==1]
                self.non_malig_df = self.fulldata[self.fulldata[label_col] ==0]
                self.min_count = min(len(self.malig_df), len(self.non_malig_df))
                non_malig_df_balanced = self.non_malig_df.sample(n=self.min_count)

                # Concatenate back
                self.df = pd.concat([self.malig_df, non_malig_df_balanced]).sample(frac=1).reset_index(drop=True)
                if aug_dir != None:
                    self.SOURCE_IDX = self.df.columns.get_loc("source")
            else:
                self.df = self.fulldata

            if self.balanced:
                self.skin_idx = self.df.columns.get_loc("skin_tone")
            
            self.img_dir = img_dir
            self.aug_dir = aug_dir
            self.non_malignant_transform = non_malignant_transform
            self.malignant_transform = malignant_transform

        def rebalance(self):
            if not self.train:
                return
            
            if (self.balanced):
                malig_skin_tones = set(self.malig_df["skin_tone"].unique())
                nonmalig_skin_tones = set(self.non_malig_df["skin_tone"].unique())
                
                exclusive_nonmalig_skin_tones = nonmalig_skin_tones - malig_skin_tones
                extra_sample_frac = 0.05  # 5% of total non_malig_df

                # Filter only the exclusive skin tones
                nonmalig_extras = self.non_malig_df[self.non_malig_df["skin_tone"].isin(exclusive_nonmalig_skin_tones)]
                
                # Sample a small subset
                nonmalig_extras_sampled = nonmalig_extras.sample(int(extra_sample_frac*self.min_count), replace=True)
                
                # Get skin_tone distribution in malignant data
                malig_skin_dist = self.malig_df["skin_tone"].value_counts(normalize=True)

                # Sample from non-malignant data using the same proportions
                non_malig_df_balanced = (
                    self.non_malig_df.groupby("skin_tone", group_keys=False)
                    .apply(lambda x: x.sample(int((1-extra_sample_frac)*self.min_count * malig_skin_dist.get(x.name, 0)), replace=True))
                )

                # Combine and shuffle
                self.df = pd.concat([self.malig_df, non_malig_df_balanced]).sample(frac=1).reset_index(drop=True)
            else:
                non_malig_df_balanced = self.non_malig_df.sample(n=self.min_count)
                # Concatenate back
                self.df = pd.concat([self.malig_df, non_malig_df_balanced]).sample(frac=1).reset_index(drop=True)



        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):
            if(self.aug_dir == None):
                img_path = os.path.join(self.img_dir, self.df.iloc[idx, 0] + '.jpg')

                image = Image.open(img_path).convert('RGB')

                if self.df.iloc[idx, MALIG_IDX] == 0 and self.non_malignant_transform:
                    image = self.non_malignant_transform(image)
                elif self.df.iloc[idx, MALIG_IDX] == 1 and self.malignant_transform:
                    image = self.malignant_transform(image)

                return image, int(self.df.iloc[idx, MALIG_IDX]), self.df.iloc[idx, 0]
            else:
                if(self.df.iloc[idx, self.SOURCE_IDX] == 0):

                    img_path = os.path.join(self.img_dir, self.df.iloc[idx, 0] + '.jpg')

                    image = Image.open(img_path).convert('RGB')

                    if self.df.iloc[idx, MALIG_IDX] == 0 and self.non_malignant_transform:
                        image = self.non_malignant_transform(image)
                    elif self.df.iloc[idx, MALIG_IDX] == 1 and self.malignant_transform:
                        image = self.malignant_transform(image)

                    return image, int(self.df.iloc[idx, MALIG_IDX]), self.df.iloc[idx, 0] 
                else:
                    img_path = os.path.join(self.aug_dir, self.df.iloc[idx, 0] + '.jpg')

                    image = Image.open(img_path).convert('RGB')

                    if self.df.iloc[idx, MALIG_IDX] == 0 and self.non_malignant_transform:
                        image = self.non_malignant_transform(image)
                    elif self.df.iloc[idx, MALIG_IDX] == 1 and self.malignant_transform:
                        image = self.malignant_transform(image)

                    return image, int(self.df.iloc[idx, MALIG_IDX]), self.df.iloc[idx, 0] 



    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet means
                            std=[0.229, 0.224, 0.225])    # ImageNet stds
    ])

    malignant_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    val_dataset = ISICDataset(
        csv_file='val_labels.csv',
        img_dir=TEST_DIR[dir],
        non_malignant_transform=transform,
        malignant_transform=transform
    )

    train_dataset = ISICDataset(
        csv_file='train_labels.csv',
        img_dir=TRAIN_DIR[dir],
        non_malignant_transform=malignant_transform,
        malignant_transform=malignant_transform,
        aug_dir = None, #AUG_DIR
        train_bool = True,
        Balance= False
    )


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Training on {'cuda' if torch.cuda.is_available() else 'cpu'}")

    for model_name in MODELS:
        resnet = initialize_model(model_name=model_name, num_classes=1)

        optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=1e-4)
        criterion = FocalLoss(alpha=0.9, gamma=2.0, reduction='mean').to(device)
        #criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([1.0]).to(device))

        # switch to this if we're doing more than malignant/not-malignant
        #criterion = nn.CrossEntropyLoss()

        resnet.to(device)

        with open(f"training_log_{model_name}_{DATA_NAMES[dir]}.csv", "w") as f:
            f.write("epoch,train_loss,val_loss,val_tn,val_fn,val_tp,val_fp,val_accuracy,val_auc,pauc,f1score\n")
            start_time = time.time()
            for epoch in range(EPOCHS):
                train_dataset.rebalance()
                dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
                valloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

                print(f"Training set size: {len(dataloader.dataset)}")
                print(f"Validation set size: {len(valloader.dataset)}")

                resnet.train()
                avgloss = 0.0
                print(f"EPOCH: {epoch + 1}")
                for images, labels, _ in dataloader:
                    images, labels = images.to(device), labels.float().unsqueeze(1).to(device)

                    optimizer.zero_grad()
                    outputs = resnet(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    avgloss += loss.item() * images.size(0)
                avgloss /= len(dataloader.dataset)

                resnet.eval()
                val_loss = 0.0
                total = 0
                false_negative = 0
                false_positive = 0
                true_negative = 0
                true_positive = 0
                all_labels = []
                all_probs = []  # Store probabilities instead of binary predictions
                image_ids_list = []  # Collect image IDs for submission

                with torch.no_grad():
                    for images, labels, image_ids in valloader:  # Ensure dataset returns image IDs
                        images = images.to(device)
                        labels = labels.to(device).float().unsqueeze(1)
                        
                        # Forward pass
                        outputs = resnet(images)
                        loss = criterion(outputs, labels)
                        val_loss += loss.item() * images.size(0)

                        # Get probabilities (before thresholding)
                        probabilities = torch.sigmoid(outputs).cpu().numpy().flatten()
                        all_probs.extend(probabilities)
                        
                        # Get binary predictions for confusion matrix
                        preds = (torch.sigmoid(outputs) > 0.8).float()
                        preds = preds.cpu().numpy().flatten()
                        
                        # Track image IDs for submission
                        image_ids_list.extend(image_ids)  # Assuming image_ids are strings
                        
                        # Update confusion matrix
                        for p, l in zip(preds, labels.cpu().numpy()):
                            if p == 0 and l == 0:
                                true_negative += 1
                            elif p == 0 and l == 1:
                                false_negative += 1
                            elif p == 1 and l == 0:
                                false_positive += 1
                            elif p == 1 and l == 1:
                                true_positive += 1
                        total += labels.size(0)
                        all_labels.extend(labels.cpu().numpy().flatten())

                # Calculate metrics
                avg_val_loss = val_loss / len(valloader.dataset)
                accuracy = (true_positive + true_negative) / total
                p_auc = roc_auc_score(all_labels, all_probs)  # Use probabilities for AUC
                f1score = f1_score(all_labels, [1 if x > 0.5 else 0 for x in all_probs])

                # Create submission DataFrame
                submission_df = pd.DataFrame({
                    'isic_id': image_ids_list,
                    'prediction': all_probs
                })

                solution_df = pd.DataFrame({
                    'isic_id': image_ids_list,
                    'is_malignant': all_labels
                })

                # Calculate pAUC using the competition metric
                try:
                    pAUC = score(
                        solution=solution_df,
                        submission=submission_df,
                        row_id_column_name='isic_id'  # Must match your column name
                    )
                except ParticipantVisibleError as e:
                    print(f"Scoring Error: {e}")
                    pAUC = -1  # Handle invalid submissions

                if (epoch+1) % 5 == 0:
                    torch.save(resnet.state_dict(), f"resnet50_{epoch}_{DUPE}_{FREEZE}_pos_weight(2).pth")
                
                print(f"Epoch [{epoch+1}/{EPOCHS}] "
                    f"Train Loss: {avgloss:.4f} "
                    f"Val Loss: {avg_val_loss:.4f}\n"
                    f"Val TN: {true_negative} FN: {false_negative} \n"
                    f"TP: {true_positive} FP: {false_positive}\n"
                    f"Val Accuracy: {accuracy:.4f}\n"
                    f"AUC: {p_auc:.4f}\n"
                    f"pAUC: {pAUC:.4f}\n"
                    f"f1 score: {f1score:.4f}\n")  # Add pAUC to output'
                f.write(f"{epoch+1},{avgloss},{avg_val_loss},{true_negative},{false_negative},{true_positive},{false_positive},{accuracy},{p_auc},{pAUC},{f1score}\n")
            
            
        print(f"Training time: {time.time() - start_time:.2f}s")

        with open(f"scores_{model_name}_{DATA_NAMES[dir]}.csv", "w") as f:
            f.write("isic_id,prediction,target\n")
            for id, fscore, actual in zip(image_ids_list, all_probs, all_labels):
                f.write(f"{id},{fscore},{actual}\n")

        

        def plot_roc_and_pauc(y_true, y_scores, min_tpr=0.80):
            # Flip labels and predictions (based on your scoring function)
            v_gt = abs(np.asarray(y_true) - 1)
            v_pred = -1.0 * np.asarray(y_scores)

            # Calculate ROC curve
            fpr, tpr, thresholds = roc_curve(v_gt, v_pred)

            # Compute full AUC and partial AUC
            full_auc = auc(fpr, tpr)
            max_fpr = abs(1 - min_tpr)

            # Partial AUC calculation (manual interpolation for the cutoff point)
            stop = np.searchsorted(fpr, max_fpr, "right")
            x_interp = [fpr[stop - 1], fpr[stop]]
            y_interp = [tpr[stop - 1], tpr[stop]]
            
            # Interpolated TPR at max_fpr
            interp_tpr = np.interp(max_fpr, x_interp, y_interp)
            
            # Create partial ROC arrays up to max_fpr
            pauc_fpr = np.append(fpr[:stop], max_fpr)
            pauc_tpr = np.append(tpr[:stop], interp_tpr)
            
            partial_auc = auc(pauc_fpr, pauc_tpr)

            # Plotting
            plt.figure(figsize=(8, 6))
            plt.plot(fpr, tpr, label=f"ROC curve (AUC = {full_auc:.4f})")
            
            # Shade pAUC region
            plt.fill_between(pauc_fpr, pauc_tpr, step="post", alpha=0.3, color="orange", label=f"pAUC = {partial_auc:.4f}")
            
            # Draw horizontal line at TPR = min_tpr
            plt.axhline(min_tpr, color='red', linestyle='--', label=f"Min TPR = {min_tpr}")

            # Draw vertical line at FPR = max_fpr
            plt.axvline(max_fpr, color='green', linestyle='--', label=f"Max FPR = {max_fpr:.2f}")

            plt.xlabel("False Positive Rate (FPR)")
            plt.ylabel("True Positive Rate (TPR)")
            plt.title("ROC Curve with pAUC Region Highlighted")
            plt.legend(loc="lower right")
            plt.grid(True)
            plt.savefig(f"roc_curve_{model_name}_{DATA_NAMES[dir]}.png")

        # Example usage
        plot_roc_and_pauc(all_labels, all_probs)

In [4]:
!pip install peft

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 71.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 63.4 MB/s eta 0:00:00:00:01


In [6]:
n = train_df[train_df.iloc[:, 0].isna()]

print(len(n))

48693


In [ ]:
print("hi")

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import time
from sklearn.metrics import roc_auc_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np
import pandas as pd
import pandas.api.types
from sklearn.metrics import roc_curve, auc, roc_auc_score
import torchvision
import timm
from peft import LoraConfig, get_peft_model


# params
FREEZE = True # whether or not to freeze the weights
EPOCHS = 15 # number of training epochs
DUPE = True
DUPE_COUNT = 100
#TRAIN_DIR = 'C:\\Users\\rngki\\Downloads\\train_images_hair_removed_dullrazor\\'
TRAIN_CSV = 'C:\\Users\\rngki\\Downloads\\train_dataset_with_skin_tone.csv'
TRAIN_DIR = ['./ISIC_2024_Training_Input/']
#TRAIN_CSV = 'C:\\Users\\rngki\\Downloads\\train-metadata.csv'
#TRAIN_CSV = 'C:\\Users\\rngki\\Downloads\\cleaned_styled_data\\cleaned_styled_data\\cleaned_augmented_data.csv'
#TEST_DIR = 'C:\\Users\\rngki\\Downloads\\test_images_hair_removed_dullrazor\\'
#TEST_CSV = 'C:\\Users\\rngki\\Downloads\\test_dataset_with_skin_tone.csv'

MODELS = ["efficientnet", "resnet50", "coatnet", "resnet34", "deit"]

DATA_NAMES = ['ISIC_2024_Training_Input']
TEST_DIR = ['./ISIC_2024_Training_Input/', "./ISIC_2024_Training_Input/"]
TEST_CSV = 'C:\\Users\\rngki\\Downloads\\test_dataset_with_skin_tone.csv'

"""
2024 ISIC Challenge primary prize scoring metric

Given a list of binary labels, an associated list of prediction 
scores ranging from [0,1], this function produces, as a single value, 
the partial area under the receiver operating characteristic (pAUC) 
above a given true positive rate (TPR).
https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.

(c) 2024 Nicholas R Kurtansky, MSKCC
"""



class ParticipantVisibleError(Exception):
    pass

def initialize_model(model_name, num_classes=1):
    model = None
    
    if model_name == "efficientnet":
        target_modules=[
            # final linear
            "classifier.1",
            # a couple of the deepest MBConv depthwise convs:
            "features.6.block.2.conv",  
            "features.7.block.2.conv",  
        ]

        model = torchvision.models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        if FREEZE:
            for param in model.parameters():
                param.requires_grad = False
        num_ftrs = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_ftrs, num_classes)
    elif model_name == "resnet34":
        target_modules=[
            # final classification
            "fc",
            # pick one conv in each of the last 3 residual stages:
            "layer2.0.conv1",
            "layer3.1.conv2",
            "layer4.2.conv2",
        ]
        model = torchvision.models.resnet34(weights=models.ResNet34_Weights.DEFAULT)
        if FREEZE:
            for param in model.parameters():
                param.requires_grad = False
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
    elif model_name == "resnet50":
        target_modules=[
            # final classification
            "fc",
            # pick one conv in each of the last 3 residual stages:
            "layer2.0.conv1",
            "layer3.1.conv2",
            "layer4.2.conv2",
        ]
        model = torchvision.models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        if FREEZE:
            for param in model.parameters():
                param.requires_grad = False
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
    elif model_name == "coatnet":
        target_modules=[
            # final linear
            "head.fc",
            # point‑wise & depth‑wise conv in the last stage:
            "stages.3.blocks.2.pwconv",
            "stages.3.blocks.2.conv",  
        ]

        model = timm.create_model('coatnet_0_rw_224', pretrained=True)
        if FREEZE:
            for param in model.parameters():
                param.requires_grad = False
        num_ftrs = model.head.fc.in_features
        model.head.fc = nn.Linear(num_ftrs, num_classes)
    elif model_name == "deit":
        target_modules=[
            # QKV + output projection in each of the first few heads:
            "blocks.0.attn.qkv",   "blocks.0.attn.proj",
            "blocks.1.attn.qkv",   "blocks.1.attn.proj",
            # …repeat for as many blocks as you like…
            # final classifier(s):
            "head", 
            "head_dist",         # only for the distilled variant
        ]

        model = timm.create_model('deit_base_patch16_224', pretrained=True)
        if FREEZE:
            for param in model.parameters():
                param.requires_grad = False
        num_ftrs = model.head.in_features
        model.head = nn.Linear(num_ftrs, num_classes)
    else:
        raise ValueError("Invalid model name")
    
    lora_config = LoraConfig(
        r=1,
        lora_alpha=2,
        lora_dropout=0.1,
        target_modules=target_modules,
    )

    model = get_peft_model(model, lora_config)
    
    return model.to(device)

def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str, min_tpr: float=0.80) -> float:
    '''
    2024 ISIC Challenge metric: pAUC
    
    Given a solution file and submission file, this function returns the
    the partial area under the receiver operating characteristic (pAUC) 
    above a given true positive rate (TPR) = 0.80.
    https://en.wikipedia.org/wiki/Partial_Area_Under_the_ROC_Curve.
    
    (c) 2024 Nicholas R Kurtansky, MSKCC

    Args:
        solution: ground truth pd.DataFrame of 1s and 0s
        submission: solution dataframe of predictions of scores ranging [0, 1]

    Returns:
        Float value range [0, max_fpr]
    '''
    for col in solution.columns:
        if col != 'is_malignant':
            del solution[col]
    
    for col in submission.columns:
        if col != 'prediction':
            del submission[col]

    # check submission is numeric
    if not pandas.api.types.is_numeric_dtype(submission.values):
        raise ParticipantVisibleError('Submission target column must be numeric')

    # rescale the target. set 0s to 1s and 1s to 0s (since sklearn only has max_fpr)
    v_gt = abs(solution.values.ravel()-1)
    
    # flip the submissions to their compliments
    v_pred = -1.0*submission.values.ravel()

    max_fpr = abs(1-min_tpr)

    # using sklearn.metric functions: (1) roc_curve and (2) auc
    fpr, tpr, _ = roc_curve(v_gt, v_pred, sample_weight=None)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected min_tpr in range [0, 1), got: %r" % min_tpr)
        
    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)

#     # Equivalent code that uses sklearn's roc_auc_score
#     v_gt = abs(np.asarray(solution.values)-1)
#     v_pred = np.array([1.0 - x for x in submission.values])
#     max_fpr = abs(1-min_tpr)
#     partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
#     # change scale from [0.5, 1.0] to [0.5 * max_fpr**2, max_fpr]
#     # https://math.stackexchange.com/questions/914823/shift-numbers-into-a-different-range
#     partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)
    
    return(partial_auc)

for dir in range(len(TRAIN_DIR)):
    AUG = False
    USE_SPLIT = False

    MALIGNANT = 'target'
    # MALIGNANT = 'target'
    MALIG_IDX = 1
    # MALIG_IDX = 3

    if not TEST_DIR:
        TEST_DIR = TRAIN_DIR[dir]
        USE_SPLIT = True
    if not TEST_CSV:
        TEST_CSV = TRAIN_CSV
        USE_SPLIT = True

    def balance_classes(df, label_col=MALIGNANT):
         # Separate malignant and non-malignant samples
         malig_df = df[df[label_col] == 1]
         non_malig_df = df[df[label_col] == 0]

         # Get the minority count
         min_count = min(len(malig_df), len(non_malig_df))

         # Downsample both classes to min_count (or oversample malignant if needed)
         non_malig_df_balanced = non_malig_df.sample(n=min_count, random_state=42)
         malig_df_balanced = malig_df.sample(n=min_count, replace=True, random_state=42)  # oversample if needed

         # Concatenate back
         balanced_df = pd.concat([malig_df_balanced, non_malig_df_balanced]).sample(frac=1, random_state=42).reset_index(drop=True)
         return balanced_df

    if USE_SPLIT:
        df = pd.read_csv(TRAIN_CSV)
        #df = pd.read_csv('C:\\Users\\rngki\\Downloads\\cleaned_styled_data\\cleaned_styled_data\\cleaned_augmented_data.csv')
        print(f"num malignant: {sum(df[MALIGNANT])}")

        # downsample for time
        df = df.sample(frac=.1, random_state=42)

        train_df, val_df = train_test_split(df, test_size=0.2, stratify=df[MALIGNANT], random_state=42)
    elif AUG:
        OG_train_df = pd.read_csv(TRAIN_CSV)
        AUG_train = pd.read_csv(AUG_CSV)
        # Add a source column to each
        OG_train_df["source"] = 0
        AUG_train["source"] = 1

        #SOURCE_IDX = AUG_train.columns.get_loc("source")

# Concatenate the two DataFrames
        train_df = pd.concat([OG_train_df, AUG_train], ignore_index=True)
        val_df = pd.read_csv(TEST_CSV)
    else:
        train_df = pd.read_csv(TRAIN_CSV)
        
        val_df = pd.read_csv(TEST_CSV)

    if DUPE:
        malig_df = train_df[train_df[MALIGNANT] == 1]
        malig_df = pd.concat([malig_df]*DUPE_COUNT, ignore_index=True)

        train_df = pd.concat([train_df, malig_df], ignore_index=True)

    train_df = balance_classes(train_df)
   # train_df = train_df.sample(frac=.001, random_state=42)
    #val_df = val_df.sample(frac=.1, random_state=42)

    train_df.to_csv('train_labels.csv', index=False)
    val_df.to_csv('val_labels.csv', index=False)

    print(f"Validation - Num malignant: {val_df[MALIGNANT].sum()}")
    print(f"Validation - Num benign: {len(val_df) - val_df[MALIGNANT].sum()}")
    print(f"Training - Num malignant: {train_df[MALIGNANT].sum()}")
    print(f"Training - Num benign: {len(train_df) - train_df[MALIGNANT].sum()}")

    class ISICDataset(Dataset):
        def __init__(self, csv_file, img_dir, non_malignant_transform=None, malignant_transform=None,label_col=MALIGNANT,aug_dir = None, train_bool = False):
            self.fulldata = pd.read_csv(csv_file)
            self.train = train_bool
            if self.train:
                self.malig_df = self.fulldata[self.fulldata[label_col] ==1]
                self.non_malig_df = self.fulldata[self.fulldata[label_col] ==0]
                self.min_count = min(len(self.malig_df), len(self.non_malig_df))
                non_malig_df_balanced = self.non_malig_df.sample(n=self.min_count)

                # Concatenate back
                self.df = pd.concat([self.malig_df, non_malig_df_balanced]).sample(frac=1).reset_index(drop=True)
                self.SOURCE_IDX = self.df.columns.get_loc("source")
            else:
                self.df = self.fulldata
            
            self.img_dir = img_dir
            self.aug_dir = aug_dir
            self.non_malignant_transform = non_malignant_transform
            self.malignant_transform = malignant_transform

        def rebalance(self):
            if not self.train:
                return

            non_malig_df_balanced = self.non_malig_df.sample(n=self.min_count)
            # Concatenate back
            self.df = pd.concat([self.malig_df, non_malig_df_balanced]).sample(frac=1).reset_index(drop=True)



        def __len__(self):
            return len(self.df)

        def __getitem__(self, idx):
            if(self.aug_dir == None):
                img_path = os.path.join(self.img_dir, self.df.iloc[idx, 0] + '.jpg')

                image = Image.open(img_path).convert('RGB')

                if self.df.iloc[idx, MALIG_IDX] == 0 and self.non_malignant_transform:
                    image = self.non_malignant_transform(image)
                elif self.df.iloc[idx, MALIG_IDX] == 1 and self.malignant_transform:
                    image = self.malignant_transform(image)

                return image, int(self.df.iloc[idx, MALIG_IDX]), self.df.iloc[idx, 0]
            else:
                if(self.df.iloc[idx, self.SOURCE_IDX] == 0):

                    img_path = os.path.join(self.img_dir, self.df.iloc[idx, 0] + '.jpg')

                    image = Image.open(img_path).convert('RGB')

                    if self.df.iloc[idx, MALIG_IDX] == 0 and self.non_malignant_transform:
                        image = self.non_malignant_transform(image)
                    elif self.df.iloc[idx, MALIG_IDX] == 1 and self.malignant_transform:
                        image = self.malignant_transform(image)

                    return image, int(self.df.iloc[idx, MALIG_IDX]), self.df.iloc[idx, 0] 
                else:

                    img_path = os.path.join(self.aug_dir, self.df.iloc[idx, 0] + '.jpg')

                    image = Image.open(img_path).convert('RGB')

                    if self.df.iloc[idx, MALIG_IDX] == 0 and self.non_malignant_transform:
                        image = self.non_malignant_transform(image)
                    elif self.df.iloc[idx, MALIG_IDX] == 1 and self.malignant_transform:
                        image = self.malignant_transform(image)

                    return image, int(self.df.iloc[idx, MALIG_IDX]), self.df.iloc[idx, 0] 



    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet means
                            std=[0.229, 0.224, 0.225])    # ImageNet stds
    ])

    malignant_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    val_dataset = ISICDataset(
        csv_file='val_labels.csv',
        img_dir=TEST_DIR[dir],
        non_malignant_transform=transform,
        malignant_transform=transform
    )

    train_dataset = ISICDataset(
        csv_file='train_labels.csv',
        img_dir=TRAIN_DIR[dir],
        non_malignant_transform=malignant_transform,
        malignant_transform=malignant_transform
    )

    dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    valloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

    print(f"Training set size: {len(dataloader.dataset)}")
    print(f"Validation set size: {len(valloader.dataset)}")

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Training on {'cuda' if torch.cuda.is_available() else 'cpu'}")

    for model_name in MODELS:
        resnet = initialize_model(model_name=model_name, num_classes=1)

        optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet.parameters()), lr=1e-4)
        criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([2.0]).to(device))

        # switch to this if we're doing more than malignant/not-malignant
        #criterion = nn.CrossEntropyLoss()

        resnet.to(device)

        with open(f"training_log_{model_name}_{DATA_NAMES[dir]}.csv", "w") as f:
            f.write("epoch,train_loss,val_loss,val_tn,val_fn,val_tp,val_fp,val_accuracy,val_auc,pauc,f1score\n")
            start_time = time.time()
            for epoch in range(EPOCHS):
                train_dataset.rebalance()
                dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
                valloader = DataLoader(val_dataset, batch_size=32, shuffle=True)

                resnet.train()
                avgloss = 0.0
                print(f"EPOCH: {epoch + 1}")
                for images, labels, _ in dataloader:
                    images, labels = images.to(device), labels.float().unsqueeze(1).to(device)

                    optimizer.zero_grad()
                    outputs = resnet(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()

                    avgloss += loss.item() * images.size(0)
                avgloss /= len(dataloader.dataset)

                resnet.eval()
                val_loss = 0.0
                total = 0
                false_negative = 0
                false_positive = 0
                true_negative = 0
                true_positive = 0
                all_labels = []
                all_probs = []  # Store probabilities instead of binary predictions
                image_ids_list = []  # Collect image IDs for submission

                with torch.no_grad():
                    for images, labels, image_ids in valloader:  # Ensure dataset returns image IDs
                        images = images.to(device)
                        labels = labels.to(device).float().unsqueeze(1)
                        
                        # Forward pass
                        outputs = resnet(images)
                        loss = criterion(outputs, labels)
                        val_loss += loss.item() * images.size(0)

                        # Get probabilities (before thresholding)
                        probabilities = torch.sigmoid(outputs).cpu().numpy().flatten()
                        all_probs.extend(probabilities)
                        
                        # Get binary predictions for confusion matrix
                        preds = (torch.sigmoid(outputs) > 0.5).float()
                        preds = preds.cpu().numpy().flatten()
                        
                        # Track image IDs for submission
                        image_ids_list.extend(image_ids)  # Assuming image_ids are strings
                        
                        # Update confusion matrix
                        for p, l in zip(preds, labels.cpu().numpy()):
                            if p == 0 and l == 0:
                                true_negative += 1
                            elif p == 0 and l == 1:
                                false_negative += 1
                            elif p == 1 and l == 0:
                                false_positive += 1
                            elif p == 1 and l == 1:
                                true_positive += 1
                        total += labels.size(0)
                        all_labels.extend(labels.cpu().numpy().flatten())

                # Calculate metrics
                avg_val_loss = val_loss / len(valloader.dataset)
                accuracy = (true_positive + true_negative) / total
                p_auc = roc_auc_score(all_labels, all_probs)  # Use probabilities for AUC
                f1score = f1_score(all_labels, [1 if x > 0.5 else 0 for x in all_probs])

                # Create submission DataFrame
                submission_df = pd.DataFrame({
                    'isic_id': image_ids_list,
                    'prediction': all_probs
                })

                solution_df = pd.DataFrame({
                    'isic_id': image_ids_list,
                    'is_malignant': all_labels
                })

                # Calculate pAUC using the competition metric
                try:
                    pAUC = score(
                        solution=solution_df,
                        submission=submission_df,
                        row_id_column_name='isic_id'  # Must match your column name
                    )
                except ParticipantVisibleError as e:
                    print(f"Scoring Error: {e}")
                    pAUC = -1  # Handle invalid submissions

                if (epoch+1) % 5 == 0:
                    torch.save(resnet.state_dict(), f"resnet50_{epoch}_{DUPE}_{FREEZE}_pos_weight(2).pth")
                
                print(f"Epoch [{epoch+1}/{EPOCHS}] "
                    f"Train Loss: {avgloss:.4f} "
                    f"Val Loss: {avg_val_loss:.4f}\n"
                    f"Val TN: {true_negative} FN: {false_negative} \n"
                    f"TP: {true_positive} FP: {false_positive}\n"
                    f"Val Accuracy: {accuracy:.4f}\n"
                    f"AUC: {p_auc:.4f}\n"
                    f"pAUC: {pAUC:.4f}\n"
                    f"f1 score: {f1score:.4f}\n")  # Add pAUC to output'
                f.write(f"{epoch+1},{avgloss},{avg_val_loss},{true_negative},{false_negative},{true_positive},{false_positive},{accuracy},{p_auc},{pAUC},{f1score}\n")
            
            
        print(f"Training time: {time.time() - start_time:.2f}s")

        with open(f"scores_{model_name}_{DATA_NAMES[dir]}.csv", "w") as f:
            f.write("isic_id,prediction,target\n")
            for id, fscore, actual in zip(image_ids_list, all_probs, all_labels):
                f.write(f"{id},{fscore},{actual}\n")

        

        def plot_roc_and_pauc(y_true, y_scores, min_tpr=0.80):
            # Flip labels and predictions (based on your scoring function)
            v_gt = abs(np.asarray(y_true) - 1)
            v_pred = -1.0 * np.asarray(y_scores)

            # Calculate ROC curve
            fpr, tpr, thresholds = roc_curve(v_gt, v_pred)

            # Compute full AUC and partial AUC
            full_auc = auc(fpr, tpr)
            max_fpr = abs(1 - min_tpr)

            # Partial AUC calculation (manual interpolation for the cutoff point)
            stop = np.searchsorted(fpr, max_fpr, "right")
            x_interp = [fpr[stop - 1], fpr[stop]]
            y_interp = [tpr[stop - 1], tpr[stop]]
            
            # Interpolated TPR at max_fpr
            interp_tpr = np.interp(max_fpr, x_interp, y_interp)
            
            # Create partial ROC arrays up to max_fpr
            pauc_fpr = np.append(fpr[:stop], max_fpr)
            pauc_tpr = np.append(tpr[:stop], interp_tpr)
            
            partial_auc = auc(pauc_fpr, pauc_tpr)

            # Plotting
            plt.figure(figsize=(8, 6))
            plt.plot(fpr, tpr, label=f"ROC curve (AUC = {full_auc:.4f})")
            
            # Shade pAUC region
            plt.fill_between(pauc_fpr, pauc_tpr, step="post", alpha=0.3, color="orange", label=f"pAUC = {partial_auc:.4f}")
            
            # Draw horizontal line at TPR = min_tpr
            plt.axhline(min_tpr, color='red', linestyle='--', label=f"Min TPR = {min_tpr}")

            # Draw vertical line at FPR = max_fpr
            plt.axvline(max_fpr, color='green', linestyle='--', label=f"Max FPR = {max_fpr:.2f}")

            plt.xlabel("False Positive Rate (FPR)")
            plt.ylabel("True Positive Rate (TPR)")
            plt.title("ROC Curve with pAUC Region Highlighted")
            plt.legend(loc="lower right")
            plt.grid(True)
            plt.savefig(f"roc_curve_{model_name}_{DATA_NAMES[dir]}.png")

        # Example usage
        plot_roc_and_pauc(all_labels, all_probs)